In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from sklearn.utils import shuffle
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import math
from scipy.spatial.distance import cdist

### Carrega e Normaliza os Dados

In [2]:
tracks = pd.read_csv('../tracks/tracks.csv')
tracks

,id,name,duration_ms,artists,explicit,popularity,acousticness,danceability,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature
0,spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,52213,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War...",t,0,0.004530,0.324,0.971,0.012800,0,0.3340,-3.168,1,0.0548,95.721,4
1,spotify:track:1xjFMVi8pn5VIWiqrdxAMc,promise [interlude],88888,"{""(5b0XQwyoJBFd3MwL2YxEPO,Nohidea)""}",f,0,0.833000,0.545,0.495,0.629000,11,0.3860,-13.538,1,0.1020,62.631,4
2,spotify:track:2ahiPTFTfIPs7UKqjKhJkc,Requiem - Alex Vidal Remix,564720,"{""(6WrPC9g41hy7LUnpxdy6u7,\""Alex Vidal\"")"",""(2...",f,0,0.033200,0.695,0.592,0.892000,0,0.5790,-9.588,1,0.0345,125.008,4
3,spotify:track:2z5xCMFxAo7dn6zsu6UwL9,Kaké,245356,"{""(44DhAhLWsSFvSqVXDUruVE,\""Young Paris\"")""}",t,0,0.000440,0.450,0.629,0.895000,7,0.1080,-7.059,1,0.1460,193.228,3
4,spotify:track:16axQgz02GFMiU2xKMT4qJ,F the Bull$h1t - Original Mix,315773,"{""(2ohlvFf9PBsDELdRstPtlP,\""Ferry Corsten\"")""}",f,0,0.000927,0.846,0.707,0.903000,6,0.0666,-5.949,0,0.1270,127.970,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2261611,spotify:track:6OjPR4Y7VGDBaJMSD3QA0t,Waking Up The Dawn,294333,"{""(7beSXA7nBwZ5xJ8jUa77ZL,\""Michael English\"")""}",f,2,0.109000,0.538,0.609,0.000006,5,0.0948,-7.392,1,0.0275,87.960,4
2261612,spotify:track:1nvTse7nkCr8tO8LOAell4,Here Comes the Star,202733,"{""(48YxSlb23RAaCd4RyHcV9V,\""Herman's Hermits\"")""}",f,0,0.698000,0.339,0.325,0.000005,2,0.5550,-11.445,1,0.0252,91.286,4
2261613,spotify:track:5vKVjyFJNtfdHXzzLaWbpz,You're With Me,293546,"{""(0WTXG5dSB0wpykAqPpEmFl,UnchainD)""}",f,0,0.018600,0.482,0.443,0.000345,4,0.1100,-8.622,1,0.0276,144.962,4
2261614,spotify:track:7LdoOfaFSBX9Cy2R3NGcAx,Reverse Skydiving (feat. Anabel Englund),329065,"{""(4aKZ8rfdsQeR7YSskFu9V3,\""Hot Natured\"")"",""(...",f,46,0.006410,0.915,0.461,0.059500,11,0.0771,-7.965,0,0.0505,118.013,4


In [3]:
# Músicas que vão simular as músicas do usuário a serem recomendadas
classical_music_i = tracks.index[tracks['name'] == 'Symphony No.5 in C minor, Op.67: 1. Allegro con brio'][0]
rap_music = tracks.index[tracks['name'] == 'Yoshi City'][0]
anime_music = tracks.index[tracks['name'] == 'GO!!!'][0]
kaio_music = tracks.index[tracks['name'] == 'Wrist Cry'][0]
kaio_music2 = tracks.index[tracks['name'] == 'Reptilia'][0]
davi_music = tracks.index[tracks['id'] == 'spotify:track:47qYqGPgMTh3l1PRKBHEfI'][0] #unity FatRat
paulo_music = tracks.index[tracks['name'] == 'abnormalize'][0]

In [4]:
# guarda o id e nome
music_ids = tracks[['id', 'name', 'artists']]
# remove colunas que não serão utilizadas
tracks.drop(['id', 'name', 'artists', 'explicit', 'time_signature', 'mode', 'key', 'duration_ms', 'popularity'], axis=1, inplace=True) 

In [5]:
tracks

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
0,0.004530,0.324,0.971,0.012800,0.3340,-3.168,0.0548,95.721
1,0.833000,0.545,0.495,0.629000,0.3860,-13.538,0.1020,62.631
2,0.033200,0.695,0.592,0.892000,0.5790,-9.588,0.0345,125.008
3,0.000440,0.450,0.629,0.895000,0.1080,-7.059,0.1460,193.228
4,0.000927,0.846,0.707,0.903000,0.0666,-5.949,0.1270,127.970
...,...,...,...,...,...,...,...,...
2261611,0.109000,0.538,0.609,0.000006,0.0948,-7.392,0.0275,87.960
2261612,0.698000,0.339,0.325,0.000005,0.5550,-11.445,0.0252,91.286
2261613,0.018600,0.482,0.443,0.000345,0.1100,-8.622,0.0276,144.962
2261614,0.006410,0.915,0.461,0.059500,0.0771,-7.965,0.0505,118.013


### Função de normalização usando fórmula do MinMaxScaler do SKLearn 
https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html

In [6]:
# função que recebe um pandas Series e a escala sem copiar memória
def scaler(X, min_=None, max_=None):
    if not min_:
        min_ = X.min()
    if not max_:
        max_ = X.max()
    return X.map(lambda x: (x - min_) / (max_ - min_))

### Normalização dos dados que não estão normalizados

In [7]:
#tracks['duration_ms'] = scaler(tracks['duration_ms'])
#tracks['popularity'] = scaler(tracks['popularity'], min_=0, max_=100)
#tracks['key'] = scaler(tracks['key'])
tracks['loudness'] = scaler(tracks['loudness'], min_=-60, max_=0)
tracks['tempo'] = scaler(tracks['tempo'])

In [8]:
tracks

,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
0,0.004530,0.324,0.971,0.012800,0.3340,0.875375,0.0548,0.382904
1,0.833000,0.545,0.495,0.629000,0.3860,0.715648,0.1020,0.250537
2,0.033200,0.695,0.592,0.892000,0.5790,0.776489,0.0345,0.500058
3,0.000440,0.450,0.629,0.895000,0.1080,0.815443,0.1460,0.772952
4,0.000927,0.846,0.707,0.903000,0.0666,0.832540,0.1270,0.511907
...,...,...,...,...,...,...,...,...
2261611,0.109000,0.538,0.609,0.000006,0.0948,0.810314,0.0275,0.351858
2261612,0.698000,0.339,0.325,0.000005,0.5550,0.747886,0.0252,0.365163
2261613,0.018600,0.482,0.443,0.000345,0.1100,0.791368,0.0276,0.579878
2261614,0.006410,0.915,0.461,0.059500,0.0771,0.801488,0.0505,0.472077


## Criando Modelo de Similaridade

In [9]:
class Similarity:
    def __init__(self, distance='euclidean'):
        self.distance = distance
        self.tracks = None
        
    def fit(self, tracks, id_tracks):
        self.tracks = tracks
        self.id_tracks = id_tracks
    
    def recommend(self, user_tracks, n_tracks=5):
        
        mean_track = np.mean(user_tracks, axis=0)

        dists = cdist([mean_track], self.tracks, self.distance)[0]
        k_arg_dists = np.argpartition(dists, n_tracks+1)[:n_tracks+1]
        k_ids = self.id_tracks.iloc[k_arg_dists]
        k_info = self.tracks.iloc[k_arg_dists]
        k_dists = pd.DataFrame(dists[k_arg_dists], columns=['Distâncias'], index=k_arg_dists)

        return pd.concat([k_dists, k_ids, k_info], axis=1).sort_values(by=['Distâncias'])
        
        

In [10]:
sim = Similarity(distance='euclidean')
sim.fit(tracks, music_ids)
sim.recommend([tracks.iloc[0]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
0,0.000000,spotify:track:0hJkacKTAr8GZeKwcPuUk2,Yesterday's Trash,"{""(24OGO7fr96L0azkG3pZZa2,\""Modern Life Is War...",0.004530,0.324,0.971,0.01280,0.334,0.875375,0.0548,0.382904
1251121,0.021866,spotify:track:0QvhDcLDbRInpUl0yVIa08,Godfather of Slander,"{""(5d6KI8frPEo3qGsIL8Sak2,Destruction)""}",0.000007,0.316,0.979,0.00563,0.323,0.880243,0.0615,0.373487
1702559,0.027671,spotify:track:0Lyv8rA24qURaeuQWVWPim,Funn,"{""(75klPfIVnyYcyEGaicRUSF,\""White Reaper\"")""}",0.000221,0.330,0.960,0.00059,0.329,0.859326,0.0443,0.389844
271533,0.029882,spotify:track:6VO3sTV6k4jxh5NFmokaCw,Skin Head,"{""(4FwoFiKgBKWrDftS9tm0oa,\""Spendtime Palace\"")""}",0.000932,0.311,0.960,0.00290,0.317,0.862884,0.0577,0.389112
103707,0.030399,spotify:track:1YDOeVm1Ls0RY1v0V6z9qN,Watch Your Head,"{""(5QKdSZ3P74BUn3RyudmmzE,\""The Bloody Hollies...",0.013000,0.313,0.961,0.00291,0.332,0.857369,0.0668,0.390736
355763,0.034384,spotify:track:5NhOXmEBbftFMBygv7GhZh,Bring It On Home,"{""(1lsXuaLnHnfL7GoJXKkUtH,\""The Hellacopters\"")""}",0.000043,0.315,0.975,0.00000,0.320,0.872372,0.0807,0.387948


In [11]:
sim2 = Similarity(distance='euclidean')
sim2.fit(tracks, music_ids)
sim2.recommend([tracks.iloc[classical_music_i]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
215695,0.000000,spotify:track:4Znak3UnsXjRYQDSK0oP6V,"Symphony No.5 in C minor, Op.67: 1. Allegro co...","{""(2wOqMjp9TyABvtHdOSOTUS,\""Ludwig van Beethov...",0.900,0.309,0.307,0.745,0.0929,0.764198,0.0443,0.402445
1819869,0.053293,spotify:track:061pN2sHhLNI3vAPIHVn2R,Stranger On The Shore,"{""(5bdgb81IJMXag724quaNSA,\""Acker Bilk\"")"",""(4...",0.940,0.319,0.322,0.742,0.1040,0.740200,0.0359,0.390764
1372793,0.055306,spotify:track:1pKwem2d9YXLd4LUParsid,THEME FROM LOVE STORY,"{""(1sLpCr5xThZkGHLw4H0mNE,\""101 Strings Orches...",0.900,0.289,0.315,0.716,0.1050,0.728632,0.0288,0.412557
1126406,0.065940,spotify:track:1mBIJRcbAec5EkveeYrPCa,I Got Lost,"{""(267VY6GX5LyU5c9M85ECZQ,\""Dinosaur Jr.\"")""}",0.875,0.354,0.330,0.748,0.0942,0.738983,0.0264,0.388224
1347218,0.069033,spotify:track:5GA90eh24Ro95k9tywfuHo,Unless a Grain of Wheat,"{""(3PF6OyABdvKssByiQ8Imn9,\""Forbes Ministry of...",0.886,0.359,0.292,0.738,0.1310,0.756327,0.0276,0.404209
130595,0.069542,spotify:track:42B96jvtflrB3Xo7rcJnl2,American Civil War Fantasy,"{""(6lxjjQH0jBo0mEHO5WxFnS,\""United States Coas...",0.910,0.319,0.303,0.773,0.0794,0.741124,0.0388,0.458044


In [12]:
sim3 = Similarity(distance='euclidean')
sim3.fit(tracks, music_ids)
sim3.recommend([tracks.iloc[rap_music]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
297206,0.000000,spotify:track:5Tou8lYopj9qCIFBFBomZ5,Yoshi City,"{""(67lytN32YpUxiSeWlKfHJ3,\""Yung Lean\"")""}",0.476,0.546,0.530,0.000000,0.0955,0.784637,0.211,0.512863
671452,0.060607,spotify:track:0kDEzhgtVZfYMVm7VR7f03,Hero,"{""(0fLP4yWtS7wE0S0QbI4IYG,\""Chan Dizzy\"")""}",0.436,0.563,0.539,0.000000,0.1220,0.796189,0.232,0.492210
554339,0.061630,spotify:track:2CIzsTwVjIWMh64Nw6S3Lp,Heartbroken Forever,"{""(5WF5qT6fSfyd3RC2p8qpEX,\""Tender Forever\"")""}",0.518,0.534,0.509,0.000000,0.1130,0.786131,0.217,0.479625
498801,0.061667,spotify:track:4vh8S4hTWAz3DHjysBo0ai,Cocaine,"{""(4YUcg2ee9TxMNdjhAxzZBF,Ashh)"",""(44JOYPmzHzT...",0.498,0.584,0.506,0.000028,0.1110,0.772084,0.181,0.511899
1055881,0.064300,spotify:track:4va49u7BqBLAmpu7WnyuQ1,Heartbreaker,"{""(4mwXUEKaW4ftbncf9Hi58l,Tank)""}",0.496,0.510,0.500,0.000000,0.0637,0.805677,0.217,0.519827
1731501,0.070310,spotify:track:00VWUa96Clte4uNEv963vn,Évite la,"{""(1cA379ILgR07m7XQVXyP6Q,Sneazzy)"",""(5olcgTze...",0.442,0.603,0.538,0.000049,0.1010,0.786347,0.232,0.513463


In [13]:
sim4 = Similarity(distance='euclidean')
sim4.fit(tracks, music_ids)
sim4.recommend([tracks.iloc[anime_music]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
906457,0.000000,spotify:track:0zFRHGYU1zP6q45qbils3z,GO!!!,"{""(3w2HqkKa6upwuXEULtGvnY,FLOW)""}",0.0508,0.604,0.978,0.000000,0.228,0.891595,0.0573,0.536584
1033472,0.038166,spotify:track:5X31etfs08Q3H7WenRHWoc,UNBALANCE,"{""(7Jo4alROJu9Smj8IZahiP1,Dios/signalP)""}",0.0471,0.590,0.956,0.004540,0.242,0.881752,0.0395,0.548101
1808708,0.042463,spotify:track:5XuGyBKKFrCoz4iwvm7k7s,I Hope,"{""(6KhH771vq2X2Aom91nNzpZ,FTISLAND)""}",0.0580,0.590,0.976,0.000000,0.197,0.892426,0.0641,0.559893
425695,0.043371,spotify:track:5z8ynRCdK17BWNLxL8rrfZ,Take It Like a Man,"{""(72OaDtakiy6yFqkt4TsiFt,Cher)""}",0.0678,0.613,0.950,0.000000,0.243,0.889977,0.0723,0.520019
1872005,0.043751,spotify:track:62pM9hM5FdqPuUKkZTmENp,fanfare,"{""(0udR1QMfgT37SQnL1QClTU,\""Bullet Train\"")""}",0.0426,0.613,0.943,0.000000,0.208,0.883323,0.0651,0.540000
2152808,0.044536,spotify:track:4618Zrqjfr2nqeXibDUIFJ,Make me fly - The Out In Space Story Continues,"{""(40edmg1pVhEKklKU7BCo1n,\""Timur Rodriguez\"")""}",0.0263,0.620,0.967,0.000008,0.250,0.889069,0.0475,0.516095


In [14]:
sim5 = Similarity(distance='euclidean')
sim5.fit(tracks, music_ids)
sim5.recommend([tracks.iloc[kaio_music]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
509178,0.000000,spotify:track:15TJBhID2NsMMp2etO0B03,Wrist Cry,"{""(2xvtxDNInKDV4AvGmjw6d1,Bladee)""}",0.366,0.634,0.428,0.003270,0.280,0.738906,0.0284,0.439895
985302,0.025748,spotify:track:5X7yF7u1fVGS4EyekX6ewM,"Miami, My Amy","{""(0bc7dhY6YXL35m1G0jVwuW,\""Keith Whitley\"")""}",0.354,0.646,0.441,0.000002,0.290,0.746592,0.0272,0.433999
754733,0.035643,spotify:track:1OJpQrusmejmwnvsFQqF8c,Annie,"{""(05Pp7SeRkvE236OxUF2qmo,Yarn)""}",0.341,0.645,0.443,0.000027,0.290,0.752599,0.0276,0.440767
2228514,0.038462,spotify:track:3zyUs7o9iKVhb1xuSSi0t8,Compañero de Mi Alma,"{""(3NUe53SAgcR19GtgfSKTpW,\""Óscar Medina\"")""}",0.374,0.646,0.413,0.000000,0.265,0.711628,0.0302,0.431942
1906283,0.040308,spotify:track:0Avs2CCYWfhanvCMc5qtnK,Friends - Acoustic,"{""(6jO0ArPsbU6gg4GehuCjoF,\""Eliza and the Bear...",0.331,0.633,0.429,0.000006,0.299,0.743912,0.0293,0.440395
1527611,0.043744,spotify:track:0DT4o6GdCw5Uxqb5EDIqSf,Don't Try To Explain,"{""(6iDaoPZVgxrTkndDCisX8F,\""Keb' Mo'\"")""}",0.386,0.649,0.452,0.000469,0.278,0.762904,0.0393,0.442339


In [15]:
sim6 = Similarity(distance='euclidean')
sim6.fit(tracks, music_ids)
sim6.recommend([tracks.iloc[kaio_music2]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
602094,0.000000,spotify:track:2hmibAtdObO8F4tnhLENuQ,Reptilia,"{""(0epOFNiUfyON9EYx7Tpr6V,\""The Strokes\"")""}",0.000597,0.488,0.650,0.714,0.1010,0.845463,0.0336,0.632077
1241745,0.039079,spotify:track:79NvOn5z1gmBhZMvAdaZzv,L'innocent,"{""(0iF2gedsSjoVHRUh5vXSdu,\""Cirque du Soleil\"")""}",0.013300,0.496,0.638,0.687,0.0973,0.827134,0.0296,0.640021
2046035,0.056278,spotify:track:1Dc4H1XvXdsRqrD6WZxYKZ,Lower Down,"{""(4zxyqU3xB0RrPX9rL9RNln,\""Lo Tom\"")""}",0.000043,0.483,0.699,0.714,0.1210,0.853781,0.0292,0.616192
899499,0.060271,spotify:track:6LVbyUVhbhhNN7uc2bqEqg,Strip The Soul,"{""(5NXHXK6hOCotCF8lvGM1I0,\""Porcupine Tree\"")""}",0.001920,0.537,0.670,0.716,0.1150,0.830538,0.0470,0.616992
1683883,0.066181,spotify:track:3JxGBrNcvunLiznL1JJzZN,Is This Real?,"{""(0sTTw3dw3EA0c7NaZnrJd2,Wipers)""}",0.003300,0.456,0.695,0.714,0.1310,0.839302,0.0324,0.651678
1838981,0.066457,spotify:track:1QlweZy3kMgha3UlkQbjMS,Take This Lying Down,"{""(2VS0tRcCbiMkelzGoZo9XP,\""The Lucksmiths\"")""}",0.002320,0.479,0.631,0.711,0.0732,0.797268,0.0264,0.603527


In [16]:
sim7 = Similarity(distance='euclidean')
sim7.fit(tracks, music_ids)
sim7.recommend([tracks.iloc[davi_music]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
1460424,0.000000,spotify:track:47qYqGPgMTh3l1PRKBHEfI,Unity,"{""(3OKg7YbOIatODzkRIbLJR4,TheFatRat)""}",0.004240,0.707,0.880,0.03790,0.0534,0.874713,0.0317,0.420034
1968698,0.041626,spotify:track:6Gd317x1MJZ378T6xbCpGI,Jacuzzi Jam - Original Mix,"{""(3uMZB1LatgHJCRe7DBtgiK,Syndaesia)""}",0.000565,0.728,0.873,0.03190,0.0659,0.872125,0.0569,0.439915
1626875,0.043683,spotify:track:5HRnngZD7Xgko5swVsuYIk,How I Feel,"{""(1fwbNA9Ymk7hPxcxx67paW,\""Brown and Gammon\"")""}",0.002050,0.698,0.889,0.03740,0.0418,0.856692,0.0541,0.447963
1569066,0.046224,spotify:track:3MYStY0UdBrk01XUDfYON1,Phoenix - DJ Laszlo Remix,"{""(188VINPQh5dyNfLUkevKqf,\""Olivia Holt\"")"",""(...",0.029200,0.697,0.899,0.01050,0.0407,0.881999,0.0365,0.412001
1277894,0.046876,spotify:track:6DlwyiBUy2b0JQWd6CLAQa,Love To Love You Baby - Giorgio Moroder Feat. ...,"{""(2eogQKWWoohI3BSnoG7E2U,\""Donna Summer\"")"",""...",0.019000,0.710,0.878,0.00499,0.0623,0.853303,0.0458,0.432158
887540,0.047079,spotify:track:5TYIzi4Oa1fiRpDX0mklLN,Hunter - Made in June Remix,"{""(4sTQVOfp9vEMCemLw50sbu,Galantis)"",""(0EJYJn2...",0.005150,0.719,0.865,0.00709,0.0827,0.871987,0.0356,0.424062


In [17]:
sim8 = Similarity(distance='euclidean')
sim8.fit(tracks, music_ids)
sim8.recommend([tracks.iloc[paulo_music]])

,Distâncias,id,name,artists,acousticness,danceability,energy,instrumentalness,liveness,loudness,speechiness,tempo
125453,0.000000,spotify:track:3GlE0YNBMDK1c3WXVPxiUW,abnormalize,"{""(00DuPiLri3mNomvvM3nZvU,\""Ling tosite sigure...",0.000006,0.352,0.889,0.157,0.0529,0.862191,0.0714,0.350222
100662,0.044577,spotify:track:7vR64t3B1hWfGYt9U19y1m,Animated Violence,"{""(3qYfqdVwX0fil71onLpLkh,\""Thee Oh Sees\"")""}",0.000101,0.358,0.906,0.172,0.0647,0.838008,0.0606,0.325801
103561,0.058122,spotify:track:0w5r7EDZm8fSjv2lyTIcTO,Elegance In Aggression,"{""(4x7qUR64FFfUx7bteNFPwY,\""Ignominious Incarc...",0.000298,0.330,0.929,0.140,0.0359,0.868044,0.0476,0.360963
1684462,0.061357,spotify:track:3EA66S5tPFBfmOkiol54bE,Velvet Dreams,"{""(3hiYKNf8gJkrqjdRPVLnTC,Genitorturers)""}",0.000307,0.371,0.891,0.170,0.0579,0.851424,0.0353,0.307956
1433285,0.063739,spotify:track:7H3nzSFki6rw1iYeg1bt11,Break Shins to This,"{""(3Cnkej21cJ1XgbAh98CbaS,\""Rabbit Junk\"")""}",0.000028,0.381,0.919,0.147,0.0833,0.854982,0.0896,0.380460
700257,0.064775,spotify:track:779lpqlAvxGUcWLTPUF3bA,Mira lo que dicen sobre nuestro amor,"{""(3RTAXX6KGdljBsOIupyZgT,\""Los Bunkers\"")""}",0.000147,0.381,0.878,0.165,0.0787,0.842367,0.0323,0.374355
